Now I will perform topic modeling by using gensim

In [2]:
import nltk  # Import the nltk module

nltk.download('wordnet')  

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /Users/lulu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Step 1: Read the preprocessed text from a file
# Assuming each line in the file is a separate document and words are separated by spaces
file_path = "../data/deepseek_output_lemmas.txt"  

# Read the file and create a list of tokenized documents (list of words)
with open(file_path, mode="r", encoding="utf-8") as file:
    text_list = [line.strip().split() for line in file.readlines()]  # Tokenize each line by space

# Step 2: Create a Dictionary
dictionary = corpora.Dictionary(text_list)

# Step 3: Create a Bag-of-Words Corpus
corpus = [dictionary.doc2bow(text) for text in text_list]

# Step 4: Train the LDA Model
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=20)

# Step 5: Print Topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

# Optional: Evaluate coherence score (optional but recommended)
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=text_list, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()
print("Coherence Score: ", coherence_score)


(0, '0.028*"technology" + 0.019*"cost" + 0.015*"datum" + 0.011*"fall" + 0.011*"chatbot" + 0.011*"system" + 0.011*"giant" + 0.010*"way" + 0.010*"export" + 0.010*"Liang"')
(1, '0.023*"lead" + 0.021*"build" + 0.020*"DeepSeeks" + 0.017*"week" + 0.017*"firm" + 0.016*"question" + 0.016*"Nvidia" + 0.015*"need" + 0.014*"Monday" + 0.013*"work"')
(2, '0.030*"market" + 0.017*"new" + 0.016*"investor" + 0.015*"world" + 0.015*"app" + 0.014*"spend" + 0.013*"startup" + 0.013*"day" + 0.012*"investment" + 0.010*"powerful"')
(3, '0.052*"tech" + 0.036*"chip" + 0.027*"like" + 0.021*"government" + 0.018*"industry" + 0.015*"call" + 0.013*"concern" + 0.011*"think" + 0.011*"include" + 0.011*"people"')
(4, '0.029*"year" + 0.023*"power" + 0.022*"stock" + 0.018*"OpenAI" + 0.018*"billion" + 0.014*"american" + 0.014*"Meta" + 0.013*"open" + 0.012*"intelligence" + 0.012*"United"')
Coherence Score:  0.8193175667658913


In [15]:
# Import necessary libraries
import pandas as pd
import numpy as np
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel

# Load the lemmatized data from the text file
file_path = "../data/deepseek_output_lemmas.txt"

# Read the file - assuming each line contains tokens for one document
with open(file_path, 'r', encoding='utf-8') as file:
    # Read all lines and strip whitespace
    lines = [line.strip() for line in file.readlines()]
    
    # Convert each line to a list of tokens (assuming tokens are space-separated)
    tokens_list = [line.split() for line in lines if line]  # Skip empty lines

# Create a DataFrame with the tokens
df = pd.DataFrame({'tokens': tokens_list})

# Print basic info about the loaded data
print(f"Loaded {len(df)} documents")
print(f"Sample document tokens: {df['tokens'][0][:10]}...")  # Show first 10 tokens of first doc

# Create a dictionary and a corpus from the tokenized data
dictionary = Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Print dictionary and corpus info to check if everything looks fine
print(f"\nDictionary size: {len(dictionary)} unique tokens")
print(f"Corpus size: {len(corpus)} documents")
print(f"First document in corpus: {corpus[0][:5]}...")  # Show first 5 token-id pairs

Loaded 20577 documents
Sample document tokens: ['UK']...

Dictionary size: 4118 unique tokens
Corpus size: 20577 documents
First document in corpus: [(0, 1)]...


In [9]:
# Add irrelevant words to stop_words
additional_stopwords = {"said", "u", "company", "deepseeks", "tech","say","china","chinese"}
stop_words = set(stopwords.words('english')).union(additional_stopwords)

def preprocess(text):
    tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation/numbers
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return tokens


In [11]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# Create a dictionary and a corpus from the tokenized data
dictionary = Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Print the dictionary and corpus to check if everything looks fine
print(dictionary.token2id)  # Check word-to-ID mapping
print(corpus[:2])  # Check corpus (list of word counts per document)


NameError: name 'df' is not defined

In [17]:
# Train the LDA model
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=30)

# Print the topics
topics = lda_model.print_topics(num_words=5)  # Print the top 5 words for each topic
for topic in topics:
    print(topic)


(0, '0.028*"year" + 0.019*"DeepSeeks" + 0.018*"industry" + 0.017*"new" + 0.016*"world"')
(1, '0.028*"technology" + 0.015*"week" + 0.011*"concern" + 0.011*"system" + 0.010*"fall"')
(2, '0.034*"chip" + 0.021*"lead" + 0.021*"build" + 0.021*"government" + 0.017*"billion"')
(3, '0.030*"market" + 0.022*"power" + 0.016*"question" + 0.016*"app" + 0.014*"american"')
(4, '0.051*"tech" + 0.027*"like" + 0.021*"stock" + 0.020*"cost" + 0.019*"OpenAI"')


In [19]:
coherence_model = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass', dictionary=dictionary)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score (u_mass): {coherence_score}")


Coherence Score (u_mass): -21.825769216702447


In [21]:
# Calculate coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['tokens'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'Coherence Score: {coherence_lda}')


Coherence Score: 0.8191685080609645


In [ ]:
#I want to remove the word deepseek ,ai,china,since they are meaningless, what do you think? I acuually expect topic like "tech""politic""economy"words like "deepseek," "ai," and "china" may dominate the topics and make them overly specific to certain aspects of your dataset, especially if they don't align with the broader themes you're aiming for, like tech, politics, and economy.
#If you're looking to obtain more general topics like tech, politics, and economy, removing these highly frequent and specific words can definitely help the model focus on more meaningful, diverse, and interpretable topics.
#Modify the Preprocessing to Exclude Specific Words:
#To ensure words like "deepseek," "ai," and "china" are excluded from the topics, you can add them to a custom stopwords list and remove them during the preprocessing stage.
custom_stopwords = stop_words.union({"deepseek", "ai", "china", "chinese", "openai"})

def preprocess(text):
    tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation/numbers
    tokens = [word for word in tokens if word not in custom_stopwords]  # Remove custom stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return tokens
